#### Import Packages

In [37]:
from cloudvolume import CloudVolume
import numpy as np
import os
import random
import requests
import tifffile
import matplotlib.pyplot as plt
import fsspec
import zarr
import dask.array as da

In [38]:
cwd = os.getcwd()
print(cwd)

/Users/suhasmathey/HHMI_DataEng/Electron_Microscopy-Data_Engineering


#### FlyEM Hemibrain Dataset - Neuroglancer

In [39]:
volume = CloudVolume('precomputed://gs://neuroglancer-janelia-flyem-hemibrain/v1.0/segmentation', use_https=True, parallel=True)

In [40]:
print("FlyEM Hemibrain - Volume size (z, y, x):", volume.bounds.maxpt)

FlyEM Hemibrain - Volume size (z, y, x): [34432 39552 41408]


In [41]:
crop_shape = np.array([1000, 1000, 1000])
max_start = volume.bounds.maxpt - crop_shape
start = np.array([random.randint(0, max_start[0]),
                  random.randint(0, max_start[1]),
                  random.randint(0, max_start[2])])
end = start + crop_shape

In [42]:
img = volume[start[0]:end[0], start[1]:end[1], start[2]:end[2]]

Decompressing: 100%|██████████| 4913/4913 [00:00<00:00, 121783.47it/s]


In [ ]:
img.shape

In [43]:
print(volume.info.keys())

dict_keys(['@type', 'data_type', 'num_channels', 'scales', 'type', 'mesh', 'segment_name_map', 'segment_properties'])


In [44]:
print("Types: ", volume.info['@type'])
print("Number of Channels: ", volume.info['num_channels'])
print("scale:", volume.info['scales'][1])
print("Data type:", volume.info['data_type'])
print("Segment Properties:", volume.info['segment_properties'])
print("Type:", volume.info['type'])
print("Segment Name Map:", volume.info['segment_name_map'])
print("Mesh:", volume.info['mesh'])


Types:  neuroglancer_multiscale_volume
Number of Channels:  1
scale: {'chunk_sizes': [[64, 64, 64]], 'compressed_segmentation_block_size': [8, 8, 8], 'encoding': 'compressed_segmentation', 'key': '16.0x16.0x16.0', 'resolution': [16, 16, 16], 'sharding': {'@type': 'neuroglancer_uint64_sharded_v1', 'data_encoding': 'gzip', 'hash': 'identity', 'minishard_bits': 6, 'minishard_index_encoding': 'gzip', 'preshift_bits': 9, 'shard_bits': 12}, 'size': [17216, 19776, 20704]}
Data type: uint64
Segment Properties: segment_properties
Type: segmentation
Segment Name Map: names
Mesh: mesh


### IDR Image Data

In [ ]:
# URL_XY = f"https://idr.openmicroscopy.org/webclient/?show=project-1161#"
# U2OS_Data = requests.get(URL_XY)

In [ ]:
# with open("U2OS_Data.ome.tif", "wb") as f:
#     f.write(U2OS_Data.content)

In [ ]:
# stack = tifffile.imread("U2OS_Data.ome.tif")
# print("Stack shape:", stack.shape)
# https://idr.openmicroscopy.org/webclient/?show=dataset-10740#

### Mouse nucleus accumbens - OpenOrganelle

In [64]:
store = fsspec.get_mapper('s3://janelia-cosem-datasets/jrc_mus-nacc-2/jrc_mus-nacc-2.zarr', anon=True)
group = zarr.open(store, mode='r')

In [65]:
print(type(group))

<class 'zarr.hierarchy.Group'>


In [66]:
print(list(group.group_keys()))

['recon-2', 'recon-2']


In [67]:
print(group.tree())

/
 ├── recon-2
 │   ├── em
 │   │   ├── fibsem-int16
 │   │   │   ├── s0 (564, 2520, 2596) int16
 │   │   │   ├── s0 (564, 2520, 2596) int16
 │   │   │   ├── s1 (282, 1260, 1298) int16
 │   │   │   ├── s1 (282, 1260, 1298) int16
 │   │   │   ├── s2 (141, 630, 649) int16
 │   │   │   ├── s2 (141, 630, 649) int16
 │   │   │   ├── s3 (70, 315, 324) int16
 │   │   │   ├── s3 (70, 315, 324) int16
 │   │   │   ├── s4 (35, 157, 162) int16
 │   │   │   ├── s4 (35, 157, 162) int16
 │   │   │   ├── s5 (17, 78, 81) int16
 │   │   │   ├── s5 (17, 78, 81) int16
 │   │   │   ├── s6 (8, 39, 40) int16
 │   │   │   ├── s6 (8, 39, 40) int16
 │   │   │   ├── s7 (4, 19, 20) int16
 │   │   │   ├── s7 (4, 19, 20) int16
 │   │   │   ├── s8 (2, 9, 10) int16
 │   │   │   └── s8 (2, 9, 10) int16
 │   │   └── fibsem-int16
 │   │       ├── s0 (564, 2520, 2596) int16
 │   │       ├── s0 (564, 2520, 2596) int16
 │   │       ├── s1 (282, 1260, 1298) int16
 │   │       ├── s1 (282, 1260, 1298) int16
 │   │       ├── 

In [26]:
zdata = group['recon-2/em/fibsem-int16/s0']

In [27]:
print("Shape:", zdata.shape)

Shape: (564, 2520, 2596)


In [49]:
attrs = print(group['recon-2']['em']['fibsem-int16'].attrs.asdict())

{'multiscales': [{'axes': [{'name': 'z', 'type': 'space', 'unit': 'nanometer'}, {'name': 'y', 'type': 'space', 'unit': 'nanometer'}, {'name': 'x', 'type': 'space', 'unit': 'nanometer'}], 'coordinateTransformations': [{'scale': [1.0, 1.0, 1.0], 'type': 'scale'}], 'datasets': [{'coordinateTransformations': [{'scale': [2.96, 4.0, 4.0], 'type': 'scale'}, {'translation': [0.0, 0.0, 0.0], 'type': 'translation'}], 'path': 's0'}, {'coordinateTransformations': [{'scale': [5.92, 8.0, 8.0], 'type': 'scale'}, {'translation': [1.48, 2.0, 2.0], 'type': 'translation'}], 'path': 's1'}, {'coordinateTransformations': [{'scale': [11.840000000000002, 16.0, 16.0], 'type': 'scale'}, {'translation': [4.4399999999999995, 6.0, 6.0], 'type': 'translation'}], 'path': 's2'}, {'coordinateTransformations': [{'scale': [23.68, 32.0, 32.0], 'type': 'scale'}, {'translation': [10.36, 14.0, 14.0], 'type': 'translation'}], 'path': 's3'}, {'coordinateTransformations': [{'scale': [47.36, 64.0, 64.0], 'type': 'scale'}, {'tra

In [59]:
labels = print(group['recon-2']['labels']['inference']['segmentations']['nuc'].attrs.asdict())

{'cellmap': {'annotation': {'annotation_type': {'encoding': {'absent': 0, 'unknown': 255}, 'type': 'instance_segmentation'}, 'class_name': 'nuc'}}, 'multiscales': [{'axes': [{'name': 'z', 'type': 'space', 'unit': 'nanometer'}, {'name': 'y', 'type': 'space', 'unit': 'nanometer'}, {'name': 'x', 'type': 'space', 'unit': 'nanometer'}], 'coordinateTransformations': [{'scale': [1.0, 1.0, 1.0], 'type': 'scale'}], 'datasets': [{'coordinateTransformations': [{'scale': [23.68, 32.0, 32.0], 'type': 'scale'}, {'translation': [10.36, 14.0, 14.0], 'type': 'translation'}], 'path': 's0'}, {'coordinateTransformations': [{'scale': [47.36, 64.0, 64.0], 'type': 'scale'}, {'translation': [22.2, 30.0, 30.0], 'type': 'translation'}], 'path': 's1'}, {'coordinateTransformations': [{'scale': [94.72, 128.0, 128.0], 'type': 'scale'}, {'translation': [45.879999999999995, 62.0, 62.0], 'type': 'translation'}], 'path': 's2'}], 'name': '', 'version': '0.4'}]}


In [48]:
print(list(attrs.keys()))

AttributeError: 'NoneType' object has no attribute 'keys'